# Install Dependencies

In [2]:
!apt-get install pkg-config lua5.1 build-essential libav-tools git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
git is already the newest version (1:2.14.1-1ubuntu4).
pkg-config is already the newest version (0.29.1-0ubuntu2).
libav-tools is already the newest version (7:3.3.4-2).
lua5.1 is already the newest version (5.1.5-8.1build1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [1]:
!pip install tqdm pydot losswise gym-retro JSAnimation retrowrapper
!pip install -U git+git://github.com/frenchie4111/dumbrain.git
!pip install --upgrade IPython
!git clone --recursive https://github.com/openai/retro-contest.git

  Cloning git://github.com/frenchie4111/dumbrain.git to /tmp/pip-req-build-rvuvps8h
  Running setup.py bdist_wheel for dumbrain ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-7kgswu63/wheels/50/8e/6f/47c68c95113aa8c02ac02bde75673ace7c3d3636842c75fcb6
Successfully built dumbrain
Requirement already up-to-date: IPython in /usr/local/lib/python3.6/dist-packages (6.4.0)
Requirement not upgraded as not directly required: pygments in /usr/local/lib/python3.6/dist-packages (from IPython) (2.1.3)
Requirement not upgraded as not directly required: pexpect; sys_platform != "win32" in /usr/local/lib/python3.6/dist-packages (from IPython) (4.5.0)
Requirement not upgraded as not directly required: prompt-toolkit<2.0.0,>=1.0.15 in /usr/local/lib/python3.6/dist-packages (from IPython) (1.0.15)
Requirement not upgraded as not directly required: decorator in /usr/local/lib/python3.6/dist-packages (from IPython) (4.3.0)
Requirement not upgraded as not directly required: setuptools>=18.5 

In [3]:
!git clone --recursive https://github.com/openai/retro-contest.git
!pip install "retro-contest/support"

fatal: destination path 'retro-contest' already exists and is not an empty directory.
Processing ./retro-contest/support
  Running setup.py bdist_wheel for retro-contest-support ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-gek4esdy/wheels/25/cf/a7/020c7dbee530e89c7461c515d3694f2e44ed2c74bab9aed8bc
Successfully built retro-contest-support
  Found existing installation: retro-contest-support 0.1.1
    Uninstalling retro-contest-support-0.1.1:
      Successfully uninstalled retro-contest-support-0.1.1


In [0]:
%matplotlib inline
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import HTML

In [0]:
def displayVideo( file ):
    return HTML(f'<video width="320" height="240" controls><source src={file} type="video/mp4"></video>')

In [4]:
# Don't run this command unless you already own the games, otherwise you are pirating :)
!python -m dumbrain.rl.retro_contest.install_games

Directory data/roms/ didn't exist, making
3694592it [00:00, 4582620.74it/s]                                               
100%|███████████████████████████| 5767168/5767168 [00:00<00:00, 91581219.07it/s]
Importing SonicAndKnuckles3-Genesis
Importing SonicTheHedgehog-Genesis
Importing SonicTheHedgehog2-Genesis
Imported 3 games


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [17]:
import retrowrapper
from retro_contest.local import make
import retro

retrowrapper.set_retro_make( make )

print( make, retrowrapper.RetroWrapper.retro_make_func )

<function make at 0x7f28884542f0> <function make at 0x7f28884542f0>


# Test Retro

In [0]:
!mkdir -p data/record/


env = retrowrapper.RetroWrapper(
    game='SonicTheHedgehog2-Genesis', 
    state='MetropolisZone.Act1'
)

In [0]:
obs = env.reset()
for i in range( 1000 ):
    action = env.action_space.sample()
    action[ 7 ] = 1
    env.step( action )

In [0]:
!ls data/record/

# Next

In [0]:
games = retro.list_games()
games = list( filter( lambda game: game.startswith( 'Sonic' ), games ) )
game_state_pairs = []
for game in games:
    for state in retro.list_states( game ):
        game_state_pairs.append( [ game, state ] )
len( game_state_pairs )

def getRetroEnv( game_state_idx ):
    game_state_pair = game_state_pairs[ game_state_idx ]
    print( game_state_pair )
    return retrowrapper.RetroWrapper(
        game=game_state_pair[ 0 ],
        state=game_state_pair[ 1 ],
        bk2dir='data/record/'
    )

In [0]:
class Agent():
    def nextAction( self, env, obs, prev_rew=None ):
        action_sample = env.action_space.sample()
        action_sample[ 7 ] = 1
        return action_sample
random_agent = Agent()

In [23]:
random_env = getRetroEnv( 40 )

obs = None
reward = None
obs = random_env.reset()

max_reward = 0
recording = []

for i in range( 1000 ):
    obs, reward, done, _ = random_env.step( random_agent.nextAction( env, obs, reward ) )
    max_reward = max( max_reward, reward )
    recording.append( obs )
    if done:
        obs = random_env.reset()

print( max_reward )

['SonicAndKnuckles3-Genesis', 'MarbleGardenZone.Act2']
5.957897424697876


In [24]:
!ls data/record/

SonicAndKnuckles3-Genesis-MarbleGardenZone.Act2-0000.bk2


In [25]:
!python -m retro.scripts.playback_movie data/record/SonicAndKnuckles3-Genesis-FlyingBatteryZone.Act1-0000.bk2

Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/retro/scripts/playback_movie.py", line 122, in <module>
    main()
  File "/usr/local/lib/python3.6/dist-packages/retro/scripts/playback_movie.py", line 110, in main
    emulator, m, duration = load_movie(movie)
  File "/usr/local/lib/python3.6/dist-packages/retro/scripts/playback_movie.py", line 81, in load_movie
    movie = retro.Movie(movie_file)
RuntimeError: Could not load movie


In [26]:
!ls data/record

SonicAndKnuckles3-Genesis-MarbleGardenZone.Act2-0000.bk2


In [27]:
displayVideo( 'data/record/SonicAndKnuckles3-Genesis-FlyingBatteryZone.Act1-0000.mp4' )